In [213]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.datasets import OxfordIIITPet
from torch.utils.data import DataLoader, Dataset
import numpy as np
from collections import Counter
import glob
import os
from PIL import Image
from sklearn.model_selection import train_test_split

In [214]:
# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [215]:
# Set Dataset Path
data_dir = "/content/data/oxford-iiit-pet/images"

In [216]:
# Transformation
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [217]:
# Define breed lists
cat_breeds = [
    'Abyssinian', 'Bengal', 'Birman', 'Bombay', 'British_Shorthair',
    'Egyptian_Mau', 'Maine_Coon', 'Persian', 'Ragdoll', 'Russian_Blue',
    'Siamese', 'Sphynx'
]

dog_breeds = [
    'American_Bulldog', 'American_Pit_Bull_Terrier', 'Basset_Hound', 'Beagle',
    'Boxer', 'Chihuahua', 'English_Cocker_Spaniel', 'English_Setter', 'German_Shorthaired',
    'Great_Pyrenees', 'Havanese', 'Japanese_Chin', 'Keeshond', 'Leonberger', 'Miniature_Pinscher',
    'Newfoundland', 'Pomeranian', 'Pug', 'Saint_Bernard', 'Samoyed', 'Scottish_Terrier',
    'Shiba_Inu', 'Staffordshire_Bull_Terrier', 'Wheaten_Terrier', 'Yorkshire_Terrier'
]

In [218]:
# Define Dataset class
class CustomPetDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform
        self.image_paths = glob.glob(os.path.join(image_folder, "*.jpg"))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert("RGB")

        class_name = "_".join(os.path.basename(img_path).split("_")[:-1]).lower()
        cat_breeds_lower = [name.lower() for name in cat_breeds]
        dog_breeds_lower = [name.lower() for name in dog_breeds]

        if class_name in cat_breeds_lower:
            binary_label = 0  # Cat
        elif class_name in dog_breeds_lower:
            binary_label = 1  # Dog
        else:
            print(f"⚠ Warning: Unknown breed found in filename {img_path}, skipping...")
            return None

        if self.transform:
            img = self.transform(img)

        return img, binary_label


In [219]:
# Create Datasets and Split
trainval_dataset = CustomPetDataset(data_dir, transform=transform)

# Get all valid labels first
all_labels = []
valid_indices = []
for idx in range(len(trainval_dataset)):
    sample = trainval_dataset[idx]
    if sample is not None:
        _, label = sample
        all_labels.append(label)
        valid_indices.append(idx)

# Create train/val split
train_idx, val_idx = train_test_split(
    valid_indices,
    test_size=0.2,
    stratify=[all_labels[i] for i in range(len(all_labels))],
    random_state=42
)

# Create the final datasets
train_dataset = torch.utils.data.Subset(trainval_dataset, train_idx)
val_dataset = torch.utils.data.Subset(trainval_dataset, val_idx)
test_dataset = CustomPetDataset(data_dir, transform=transform)

In [220]:
# Create Data Loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

In [221]:
# Print dataset sizes
print(f"Training size: {len(train_dataset)}")
print(f"Validation size: {len(val_dataset)}")
print(f"Test size: {len(test_dataset)}")

Training size: 5912
Validation size: 1478
Test size: 7390


In [222]:
# Load pretrained ResNet18
model = torchvision.models.resnet18(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [223]:
# Replace final layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)  # Binary output
model = model.to(device)

In [224]:
# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)

In [225]:
# Training loop
n_epochs = 5
for epoch in range(5):
    model.train()
    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.float().unsqueeze(1).to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{n_epochs}], Loss: {running_loss/len(train_loader):.4f}")


Epoch [1/10], Loss: 0.0966
Epoch [2/10], Loss: 0.0261
Epoch [3/10], Loss: 0.0308
Epoch [4/10], Loss: 0.0234
Epoch [5/10], Loss: 0.0292


In [226]:
# Evaluate
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        predicted = (torch.sigmoid(outputs) > 0.5).squeeze().long()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 99.74%
